In [1]:
import warnings
warnings.filterwarnings('ignore')
import os 
from dotenv import load_dotenv
load_dotenv()
from crewai import Agent,Task,Crew,LLM


In [10]:
serper_api_key = os.getenv("SERPER_API_KEY")

In [25]:
llm = LLM(
    model = "ollama/gemma2:2b",
    base_url = 'http://localhost:11434'
)

In [26]:
import yaml
files = {
    'agents':'config/agents.yaml',
    'tasks':'config/tasks.yaml'
}

configs = {}
for config_type , file_path in files.items():
    with open(file_path,'r') as file:
        configs[config_type] = yaml.safe_load(file)

agents_config = configs['agents']
tasks_config = configs['tasks']
    

In [27]:
from pydantic import BaseModel, Field
from typing import List

class SocialMediaPost(BaseModel):
    platform: str = Field(..., description="The social media platform where the post will be published (e.g., Twitter, LinkedIn).")
    content: str = Field(..., description="The content of the social media post, including any hashtags or mentions.")

class ContentOutput(BaseModel):
    article: str = Field(..., description="The article, formatted in markdown.")
    social_media_posts: List[SocialMediaPost] = Field(..., description="A list of social media posts related to the article.")

In [28]:
from crewai_tools import SerperDevTool, ScrapeWebsiteTool, WebsiteSearchTool

In [31]:
# ... existing code ...
from crewai_tools import MDXSearchTool
# Use MDXSearchTool with Ollama configuration instead

website_search_tool = MDXSearchTool(
    mdx='',  # Empty string for general web search
    config=dict(
        llm=dict(
            provider="ollama",
            config=dict(
                model="gemma2:2b",  # Use the same model as your main LLM
            ),
        ),
        embedder=dict(
            provider="ollama",
            config=dict(
                model="mxbai-embed-large:latest",  # Your embedding model
            ),
        ),
    )
)
# ... existing code ...

/opt/anaconda3/envs/DSenv/lib/python3.11/site-packages/embedchain/embedder/ollama.py:27: LangChainDeprecationWarning: The class `OllamaEmbeddings` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import OllamaEmbeddings``.
  embeddings = OllamaEmbeddings(model=self.config.model, base_url=config.base_url)
/opt/anaconda3/envs/DSenv/lib/python3.11/site-packages/chromadb/types.py:144: PydanticDeprecatedSince211: Accessing the 'model_fields' attribute on the instance is deprecated. Instead, you should access this attribute from the model class. Deprecated in Pydantic V2.11 to be removed in V3.0.
  return self.model_fields  # pydantic 2.x


FileNotFoundError: [Errno 2] No such file or directory: ''

In [ ]:
# Creating Agents
market_news_monitor_agent = Agent(
    config=agents_config['market_news_monitor_agent'],
    tools=[SerperDevTool(), ScrapeWebsiteTool()],
    llm=llm,
)

data_analyst_agent = Agent(
    config=agents_config['data_analyst_agent'],
    tools=[SerperDevTool(),website_search_tool],
    llm=llm,
)

content_creator_agent = Agent(
    config=agents_config['content_creator_agent'],
    tools=[SerperDevTool(), website_search_tool],
    llm=llm
)

quality_assurance_agent = Agent(
    config=agents_config['quality_assurance_agent'],
    llm=llm
)

# Creating Tasks
monitor_financial_news_task = Task(
    config=tasks_config['monitor_financial_news'],
    agent=market_news_monitor_agent
)

analyze_market_data_task = Task(
    config=tasks_config['analyze_market_data'],
    agent=data_analyst_agent
)

create_content_task = Task(
    config=tasks_config['create_content'],
    agent=content_creator_agent,
    context=[monitor_financial_news_task, analyze_market_data_task]
)

quality_assurance_task = Task(
    config=tasks_config['quality_assurance'],
    agent=quality_assurance_agent,
    output_pydantic=ContentOutput
)

# Creating Crew
content_creation_crew = Crew(
    agents=[
        market_news_monitor_agent,
        data_analyst_agent,
        content_creator_agent,
        quality_assurance_agent
    ],
    tasks=[
        monitor_financial_news_task,
        analyze_market_data_task,
        create_content_task,
        quality_assurance_task
    ],
    verbose=True,
    llm=llm
)

KeyError: 'OPENAI_API_KEY'

In [ ]:
result = content_creation_crew.kickoff(inputs={
  'subject': 'Inflation in the US and the impact on the stock market in 2024'
})

In [ ]:
import textwrap

posts = result.pydantic.dict()['social_media_posts']
for post in posts:
    platform = post['platform']
    content = post['content']
    print(platform)
    wrapped_content = textwrap.fill(content, width=50)
    print(wrapped_content)
    print('-' * 50)

In [ ]:
from IPython.display import display, Markdown
display(Markdown(result.pydantic.dict()['article']))